In [1]:
!pip install --upgrade -q kaggle 
!mkdir /root/.kaggle
import json 
token = {
    
    "username": "ifrahandleeb",
    "key": "7d32d51ce48e80317141c1a4fc7c3c23"
    
}

with open('/root/.kaggle/kaggle.json' , 'w') as config_file:

  json.dump(token , config_file)

!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d gpiosenka/100-bird-species

100% 1.81G/1.81G [00:16<00:00, 63.7MB/s]
100% 1.81G/1.81G [00:16<00:00, 119MB/s] 


In [2]:
import zipfile 

zip_ref = zipfile.ZipFile("/content/100-bird-species.zip")
zip_ref.extractall()
zip_ref.close()

In [3]:
!ls /content/birds

birds.csv  test  train	valid


In [4]:
!ls /content/birds/train 

'AFRICAN CROWNED CRANE'     'INCA TERN'
'AFRICAN FIREFINCH'	    'INDIAN BUSTARD'
 ALBATROSS		    'INDIAN PITTA'
'ALEXANDRINE PARAKEET'	    'INDIGO BUNTING'
'AMERICAN AVOCET'	     JABIRU
'AMERICAN BITTERN'	    'JAVA SPARROW'
'AMERICAN COOT'		     KAKAPO
'AMERICAN GOLDFINCH'	     KILLDEAR
'AMERICAN KESTREL'	    'KING VULTURE'
'AMERICAN PIPIT'	     KIWI
'AMERICAN REDSTART'	     KOOKABURRA
 ANHINGA		    'LARK BUNTING'
'ANNAS HUMMINGBIRD'	    'LEARS MACAW'
 ANTBIRD		    'LILAC ROLLER'
'ARARIPE MANAKIN'	    'LONG-EARED OWL'
'ASIAN CRESTED IBIS'	    'MAGPIE GOOSE'
'BALD EAGLE'		    'MALABAR HORNBILL'
'BALI STARLING'		    'MALACHITE KINGFISHER'
'BALTIMORE ORIOLE'	     MALEO
 BANANAQUIT		    'MALLARD DUCK'
'BANDED BROADBILL'	    'MANDRIN DUCK'
'BARN OWL'		    'MARABOU STORK'
'BARN SWALLOW'		    'MASKED BOOBY'
'BARRED PUFFBIRD'	    'MASKED LAPWING'
'BAR-TAILED GODWIT'	    'MIKADO  PHEASANT'
'BAY-BREASTED WARBLER'	    'MOURNING DOVE'
'BEARDED BARBET'	     MYNA
'BEARDED REEDLING'	    'NICOBAR PIGE

In [5]:
import os 

num_train = len(os.listdir("/content/birds/train"))
num_train

275

In [6]:
num_test = len(os.listdir("/content/birds/valid"))
num_test

275

In [7]:
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Conv2D , MaxPool2D , Flatten , Activation , Dense 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
tf.random.set_seed(42)

train_datagen = ImageDataGenerator(rescale = 1/255)
test_datagen = ImageDataGenerator(rescale = 1/255)
val_datagen = ImageDataGenerator(rescale = 1/255)

train_dir = "/content/birds/train/"
test_dir = "/content/birds/test/"
val_dir = "/content/birds/valid/"

train_data = train_datagen.flow_from_directory(directory = train_dir , 
                                               batch_size = 32 , 
                                               target_size = (224 , 224) ,
                                               class_mode = "categorical" )
val_data = val_datagen.flow_from_directory(directory = val_dir , 
                                           batch_size = 32 , 
                                           target_size = (224 ,224) , 
                                           class_mode = "categorical" )
test_data = test_datagen.flow_from_directory(directory = test_dir , 
                                           batch_size = 32 , 
                                           target_size = (224 ,224) , 
                                           class_mode = "categorical"
                                            
)

Found 39364 images belonging to 275 classes.
Found 1375 images belonging to 275 classes.
Found 1375 images belonging to 275 classes.


In [9]:
model = Sequential ([
                      Conv2D(10 , 3 , activation= "relu") , 
                      MaxPool2D() , 
                      Conv2D(10 , 3 , activation = "relu") , 
                      MaxPool2D() , 
                      Flatten() , 
                      Dense( 275 , activation = "softmax")
                     
])

In [10]:
model.compile( loss = "mae" , 
                optimizer = Adam() , 
                metrics = ["accuracy"])

In [11]:
history = model.fit(    train_data , 
                        epochs = 5 , 
                        steps_per_epoch = len(train_data) , 
                        validation_data = val_data ,
                        validation_steps = len(val_data) 

                        )

Epoch 1/5
 247/1231 [=====>........................] - ETA: 14:39 - loss: 0.0072 - accuracy: 0.0048

KeyboardInterrupt: ignored

In [ ]:
model = Sequential ([
                      Conv2D(10 , 3 , activation= "relu") , 
                      MaxPool2D() , 
                      Conv2D(10 , 3 , activation = "relu") , 
                      MaxPool2D() , 
                      Flatten() , 
                      Dense( 275 , activation = "softmax")
                     
])

model.compile( loss = "categorical_crossentropy" , 
                optimizer = Adam() , 
                metrics = ["accuracy"])

history = model.fit(    train_data , 
                        epochs = 5 , 
                        steps_per_epoch = len(train_data) , 
                        validation_data = val_data ,
                        validation_steps = len(val_data) 

                        )

Epoch 1/5
1231/1231 [==============================] - 1098s 891ms/step - loss: 4.3356 - accuracy: 0.1715 - val_loss: 2.9347 - val_accuracy: 0.3833
Epoch 2/5
 323/1231 [======>.......................] - ETA: 13:14 - loss: 1.9677 - accuracy: 0.5892

In [ ]:
import pandas as pd 
pd.DataFrame(history.history).plot(figsize =(10 , 7));

In [ ]:
def plot_loss_curves(history):

  loss = history.history["loss"]
  val_loss = history.history["val_loss"]

  accuracy = ["accuracy"]
  val_accuracy = ["val_accuracy"]

  epochs = range(len(history.history["loss"]))
  
  plt.figure()
  plt.plot(epochs , loss , label="training_loss")
  plt.plot(epochs , val_loss , label = "val_loss")
  plt.title("loss")
  plt.xlabel("epochs")
  plt.legend()
  
  plt.figure()
  plt.plot(epochs , loss , label = "training_accuracy")
  plt.plot(epochs , val_loss , label = "val_accuracy")
  plt.title("accuracy")
  plt.xlabel("epochs")
  plt.legend();

In [ ]:
import matplotlib.pyplot as plt

plot_loss_curves(history)


In [ ]:
model.predict(test_data)

In [ ]:
import matplotlib.image as mpimg
!wget https://raw.githubusercontent.com/Zahid8/datasets_for_cnn/master/1.jpg
bird_img = mpimg.imread(bird.jpg)
plt.imshow(bird_img)
plt.axis(False);

In [ ]:
def load_and_prep_image(filename , img_shape=224):

  img = tf.io.read_file(filename)
  img = tf.image.decode_image(img)
  img = tf.image.resize(img , size=[img_shape ,img_shape])
  img = img/225
  return img

In [ ]:
bird_img = load_and_prep_image("bird.jpg")
bird_img 

In [ ]:
model.predict(tf.expand_dims(bird_img , axis=0))

In [ ]:
import pathlib
import numpy as np
data_dir = pathlib.Path("/content/birds/train")
class_names= np.array(sorted([item.name for item in data_dir.glob("*")]))
class_names = class_names[1:]
print(class_names)

In [ ]:
pred_class = class_names[int(tf.round(pred)[0][0])]
pred_class

In [ ]:
pred = moel.(bird)
pred